# CSCI 3155 Recitation 8
March 15

### Examples

https://docs.google.com/presentation/d/1OlAu7pjBfKogQg6Z5Ykkid9FfWVH-yrSbPUCHMI-3HM/edit?usp=sharing

## Exercise
Given the following functions, change them to use continuations:

In [1]:
def doThing1(): Int = 5 + 3
def doThing2(a: Int, b: Int): Int = a + b
def doThing3(f: Int => String): Int = {
    println(f(5))
    5
}

defined function doThing1
defined function doThing2
defined function doThing3

In [2]:
// BEGIN SOLUTION
def doThing1_cont(k: Int => Int): Int = k(5 + 3)
def doThing2_cont(a: Int, b: Int, k: Int => Int): Int = k(a + b)
def doThing3_cont(f: Int => String, k: Int => Int): Int = {
    println(f(5))
    k(5)
}
// END SOLUTION

defined function doThing1_cont
defined function doThing2_cont
defined function doThing3_cont

## Exercise
Update the following expression to use the new continuation functions:

In [3]:
val x = doThing1()
val y = doThing2(x, 3)
val z = doThing3(n => "Log: " + n.toString)

Log: 5


x: Int = 8
y: Int = 11
z: Int = 5

In [4]:
// BEGIN SOLUTION
val z2 = 
    doThing1_cont(x =>
        doThing2_cont(x, 3, y =>
            doThing3_cont(n => "Log: " + n.toString, n =>
                n)))
// END SOLUTION

assert(z == z2)

Log: 5


z2: Int = 5

### Exercise: Backtracking
As with any good programing tool, we can use continuations to solve problems without putting them everywhere in our code. In this example, you will write a search function that looks for a value in a binary tree, **not** a B**S**T. Take advantage of continuations to remember where to look if something isn't found while keeping the function tail recursive.

In [5]:
sealed trait Tree
case object Empty extends Tree
case class Node(l: Tree, d: Int, r: Tree) extends Tree

def search(t: Tree, i: Int, fail_continuation: () => Boolean): Boolean =
    // BEGIN SOLUTION
    t match {
        case Empty => fail_continuation()
        case Node(l, j, r) if i == j =>
            true
        case Node(l, j, r) =>
            search(l, i, () => {
                search(r, i, () => {
                    fail_continuation()
                })
            })
    }
    //END SOLUTION


defined trait Tree
defined object Empty
defined class Node
defined function search

In [6]:
val t = Node(Empty, 10, Node(Empty, 6, Empty))
assert(search(t, 10, () => false))
assert(!search(t, 0, () => false))

t: Node = Node(Empty,10,Node(Empty,6,Empty))

### Exercise: Eval (again)

This is similar to the example in class: implement eval for the small language given below, ensuring that every function call is a tail call.

In [7]:
sealed trait Expr
case class BoolLiteral(b: Boolean) extends Expr
case class And(left: Expr, right: Expr) extends Expr
case class If(test: Expr, then: Expr, otherwise: Expr) extends Expr

def eval(e: Expr, continuation: Boolean => Boolean): Boolean =
    // BEGIN SOLUTION
    e match {
        case BoolLiteral(b) =>
            continuation(b)
        case And(left, right) =>
            eval(left, left_val =>
                 eval(right, right_val =>
                      continuation(left_val && right_val)
                     )
                )
        case If(test, then, otherwise) =>
            eval(test, test_val =>
                 if (test_val) eval(then, b => continuation(b))
                 else eval(otherwise, continuation)
                )
    }
    // END SOLUTION

defined trait Expr
defined class BoolLiteral
defined class And
defined class If
defined function eval

In [8]:
val e = If(
    And(BoolLiteral(true), BoolLiteral(true)),
    BoolLiteral(false),
    BoolLiteral(true)
)
assert(!eval(e, x => x))

e: If = If(And(BoolLiteral(true),BoolLiteral(true)),BoolLiteral(false),BoolLiteral(true))

## Exercise: Async Continuations (Callbacks)
Use the `http_get` function to request the given url and provide a continuation that will append `" lie."` to the result

In [9]:
import $ivy.`com.lihaoyi::requests:0.1.4`
import scala.concurrent.{ Future, Await }
import scala.concurrent.duration._
import scala.concurrent.ExecutionContext.Implicits.global

def http_get(url: String, k: String => String): Future[String] =
    Future(requests.get(url)).map(_.text).map(k)

import $ivy.$                            

import scala.concurrent.{ Future, Await }

import scala.concurrent.duration._

import scala.concurrent.ExecutionContext.Implicits.global


defined function http_get

In [10]:
val shakira_url = "http://www.mocky.io/v2/5c6fa3fc3400004e5f8931a6"

// BEGIN SOLUTION
val res = http_get(shakira_url, result => result + " lie.")
// END SOLUTION

shakira_url: String = "http://www.mocky.io/v2/5c6fa3fc3400004e5f8931a6"
res: Future[String] = Future(<not completed>)

In [11]:
// Run this cell if you're using the older scala kernel (from VM)
// which doesn't update the future
Await.result(res, 5000 millis)

res10: String = "Hips don't lie."